In [ ]:
import os
import numpy as np
import xarray as xr

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import cmocean

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings('ignore', category=FutureWarning)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['mathtext.fontset'] = 'cm'

We will focus on the Hunga Tonga-Hunga Ha'apa Volcano Eruption that happened the 15th of January 2022 at 4:27 am (UTC time). 

<img src="https://media.ktoo.org/wp-content/uploads/2022/01/maxar_sat1_custom-e038a29a9a579b08a82098337bf78b2505ebe6e8-830x329.jpg" width="100%">

# Wave climate in South Pacific for the month of January 2022


Here I have taken the data from CSIRO `WAVEWATCHIII` wave model for the South Pacific region, selected specific variables and clipped the output to the region of interest. Then I saved the file as a new `netcdf` file (`southpacificJan2022.nc`) that is uploaded in the Jupyter environment.

_Note: I have done that to reduce the size of the file, due to memory and space limitation required by the Binder application._

The code for this step is given here for info:

```python
    # Loading the dataset
    url = "https://data-cbr.csiro.au/thredds/dodsC/catch_all/CMAR_CAWCR-Wave_archive/CAWCR_Wave_Hindcast_aggregate/gridded/ww3.pac_4m.202201.nc"
    dataset = xr.open_dataset(url, decode_times =  True, decode_cf = True, mask_and_scale =  True)
    
    min_lon = 176.8     # lower left longitude
    min_lat = -19.5    # lower left latitude
    max_lon = 181.5     # upper right longitude
    max_lat = -16.5     # upper right latitude

    # Defining the boundaries
    lon_bnds = [min_lon, max_lon]
    lat_bnds = [min_lat, max_lat]

    # Performing the reduction
    dataset_clip = dataset.sel(latitude=slice(*lat_bnds), longitude=slice(*lon_bnds))
    
    # Set timeline
    january_data = dataset_clip.sel(time=slice('2022-01-03', '2022-01-31'))
    
    # Save tonga january
    time = dataset_clip.time.values
    latitude = dataset_clip.latitude.values
    longitude = dataset_clip.longitude.values

    dataset = xr.Dataset({
            'hs': xr.DataArray(
                        data   = dataset_clip.hs.values,   
                        dims   = ['time', 'latitude', 'longitude'],
                        coords = {'time': time, 'latitude': latitude, 'longitude':longitude},
                        attrs  = dataset_clip.hs.attrs
            ),
            't01': xr.DataArray(
                        data   = dataset_clip.t01.values,   
                        dims   = ['time', 'latitude', 'longitude'],
                        coords = {'time': time, 'latitude': latitude, 'longitude':longitude},
                        attrs  = dataset_clip.t01.attrs
                        ),
            'lm': xr.DataArray(
                        data   = dataset_clip.lm.values,   
                        dims   = ['time', 'latitude', 'longitude'],
                        coords = {'time': time, 'latitude': latitude, 'longitude':longitude},
                        attrs  = dataset_clip.lm.attrs
                        ),
            'u': xr.DataArray(
                        data   = dataset_clip.uwnd.values,   
                        dims   = ['time', 'latitude', 'longitude'],
                        coords = {'time': time, 'latitude': latitude, 'longitude':longitude},
                        attrs  = dataset_clip.uwnd.attrs
                        ),
            'v': xr.DataArray(
                        data   = dataset_clip.vwnd.values,   
                        dims   = ['time', 'latitude', 'longitude'],
                        coords = {'time': time, 'latitude': latitude, 'longitude':longitude},
                        attrs  = dataset_clip.vwnd.attrs
                        ),
            })
    
    # Save to file
    dataset.to_netcdf(path='southpacificJan2022.nc')
    dataset.close()
```

# Loading the dataset

Let's load the file:

In [ ]:
jan_data = xr.open_dataset('../../dataset/southpacificJan2022.nc', decode_times =  True, decode_cf = True, mask_and_scale =  True)
jan_data

## Temporal variations

We will now look at the model results for Tonga, first by selecting `sel` the closest point in our model to the Tonga coordinates:

In [ ]:
# Tonga point
Tonga_lat = -20.7
Tonga_lon = 184.7
Tonga = jan_data.sel(longitude=Tonga_lon, latitude=Tonga_lat, method='nearest')

Let's visualise the different variables:

+ the significant wave height

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

Tonga.hs.plot(ax=ax, x='time', lw=3)

ax.grid(True, linewidth=0.5, color="k", alpha=0.1, linestyle="-")
ax.tick_params(labelcolor="k", labelsize="medium", width=3)

plt.tight_layout()

+ the mean wave height

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

Tonga.lm.plot(ax=ax, x='time', lw=3)

ax.grid(True, linewidth=0.5, color="k", alpha=0.1, linestyle="-")
ax.tick_params(labelcolor="k", labelsize="medium", width=3)

plt.tight_layout()

+ the wave period

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

Tonga.t01.plot(ax=ax, x='time', lw=3)

ax.grid(True, linewidth=0.5, color="k", alpha=0.1, linestyle="-")
ax.tick_params(labelcolor="k", labelsize="medium", width=3)

plt.tight_layout()


<div class="alert alert-block alert-info">From the graphs above, do you see the Tonga eruption?</div>


## Tropical Cyclone Cody


<img src="https://pbs.twimg.com/tweet_video_thumb/FIr7LsFUcAApEHJ.jpg" width="80%">

### Disaster description

_Tropical Disturbance TD03F intensified into a Tropical Depression on 8 January 2022. By 10 January, it had transformed into the category 1 Tropical Cyclone Cody, bringing significant rain affecting the whole of Fiji. [...] The high level of precipitation over the Fiji group between 8-15 January has been significant and made most of the western and central divisions water-logged and highly susceptible to flooding. Extensive flooding in the Western region has also affected some communities in the Central and Eastern divisions. Water levels were slow to subside in some areas due to saturated land._


To analyse the evolution, we will first create a couple of functions to plot the evolution over time:

In [ ]:
def plotMapWave(timevar, hmax=1, umax=0.2, arrowscale=8, shaft=0.005):
    
    # Figure size
    size = (9, 10)

    dataset = jan_data.isel(time=timevar)

    mean_cur = np.sqrt(dataset.u.values**2+dataset.v.values**2)
    dataset['mean_cur'] = (('latitude', 'longitude'),mean_cur)

    # Color from cmocean
    color = cmocean.cm.dense

    # Defining the figure
    fig = plt.figure(figsize=size, facecolor='w', 
                     edgecolor='k')

    # Axes with Cartopy projection
    ax = plt.axes(projection=ccrs.PlateCarree(180))
    # and extent
    min_lon = 176.8     # lower left longitude
    min_lat = -19.5    # lower left latitude
    max_lon = 181.5     # upper right longitude
    max_lat = -16.5     # upper right latitude
    ax.set_extent([min_lon, max_lon, min_lat, max_lat], 
                  ccrs.PlateCarree())
    ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree(), alpha=0.5, 
                 xlocs=[177,178,179,180,-179])

    # Add map features with Cartopy 
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', 
                                                edgecolor='face', 
                                                facecolor='lightgray'))
    ax.coastlines(linewidth=1)

    # Plotting using Matplotlib the mean current
    cf = dataset.hs.plot( 
        transform=ccrs.PlateCarree(), cmap=color,
        vmin = 0.01, vmax = hmax,
        add_colorbar=False
    )

    # Resampling using the slice method
    resample = dataset.isel(longitude=slice(None, None, 6),
                                  latitude=slice(None, None, 6))

    # Defining the quiver plot
    quiver = resample.plot.quiver(x='longitude', y='latitude', u='u', v='v', 
                                  transform=ccrs.PlateCarree(), scale=arrowscale, width=0.0025, zorder=1)      

    # Vector options declaration
    veclenght = umax*0.5
    maxstr = '%4.2f m/s' % veclenght
    plt.quiverkey(quiver,0.07,0.07,veclenght,maxstr,labelpos='S', 
                  coordinates='axes').set_zorder(2)


    # Color bar
    cbar = fig.colorbar(cf, ax=ax, fraction=0.027, pad=0.065, 
                        orientation="horizontal")
    cbar.set_label('Significant wave height (m)', rotation=0, 
                   labelpad=5, fontsize=10)
    cbar.ax.tick_params(labelsize=8)

    plt.show()
    fig.clear()
    plt.close(fig)
    plt.clf()

def plotMapWind(timevar, umax=0.2, arrowscale=8, shaft=0.005):
    
    # Figure size
    size = (9, 10)

    dataset = jan_data.isel(time=timevar)

    mean_cur = np.sqrt(dataset.u.values**2+dataset.v.values**2)
    dataset['mean_cur'] = (('latitude', 'longitude'),mean_cur)

    # Color from cmocean
    color = cmocean.cm.speed

    # Defining the figure
    fig = plt.figure(figsize=size, facecolor='w', 
                     edgecolor='k')

    # Axes with Cartopy projection
    ax = plt.axes(projection=ccrs.PlateCarree(180))
    # and extent
    min_lon = 176.8     # lower left longitude
    min_lat = -19.5    # lower left latitude
    max_lon = 181.5     # upper right longitude
    max_lat = -16.5     # upper right latitude
    ax.set_extent([min_lon, max_lon, min_lat, max_lat], 
                  ccrs.PlateCarree())
    ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree(), alpha=0.5, 
                 xlocs=[177,178,179,180,-179])

    # Add map features with Cartopy 
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', 
                                                edgecolor='face', 
                                                facecolor='lightgray'))
    ax.coastlines(linewidth=1)

    # Plotting using Matplotlib the mean current
    cf = dataset.mean_cur.plot( 
        transform=ccrs.PlateCarree(), cmap=color,
        vmin = 0.01, vmax = umax,
        add_colorbar=False
    )

    # Resampling using the slice method
    resample = dataset.isel(longitude=slice(None, None, 6),
                                  latitude=slice(None, None, 6))

    # Defining the quiver plot
    quiver = resample.plot.quiver(x='longitude', y='latitude', u='u', v='v', 
                                  transform=ccrs.PlateCarree(), scale=arrowscale, width=0.0025, zorder=1)      

    # Vector options declaration
    veclenght = umax*0.5
    maxstr = '%4.2f m/s' % veclenght
    plt.quiverkey(quiver,0.07,0.07,veclenght,maxstr,labelpos='S', 
                  coordinates='axes').set_zorder(2)


    # Color bar
    cbar = fig.colorbar(cf, ax=ax, fraction=0.027, pad=0.065, 
                        orientation="horizontal")
    cbar.set_label('mean wind speed 10 m above sea (m/s)', rotation=0, 
                   labelpad=5, fontsize=10)
    cbar.ax.tick_params(labelsize=8)

    plt.show()
    fig.clear()
    plt.close(fig)
    plt.clf()

Now we will call the function to evaluate the wave and wind evolution over time during the cyclone:

In [ ]:
plotMapWave(40, hmax=5., umax=20., arrowscale=200, shaft=0.001)
plotMapWind(40, umax=20., arrowscale=200, shaft=0.001)

In [ ]:
plotMapWave(200, hmax=5., umax=20., arrowscale=200, shaft=0.001)
plotMapWind(200, umax=20., arrowscale=200, shaft=0.001)

In [ ]:
plotMapWave(320, hmax=5., umax=20., arrowscale=200, shaft=0.001)
plotMapWind(320, umax=20., arrowscale=200, shaft=0.001)